# Primera Implementación de Algoritmos 

### Obteniendo los datos

In [5]:
import glob
import numpy as np
import matplotlib.pyplot as plt

x=np.genfromtxt('dia.txt',delimiter=',')

#Imprimiendo, toda la matriz de datos de entrada
#print(np.shape(x))
#print(type(x))
#print(x[0:2,:])

### Ordenando los datos

In [38]:
""""
#Normalizando Columnas, descomente esto para tener una visualización de que la función min se hace sobre una sola columna, y la fila igual
val = [[30,2,3],[14,5,17],[22,8,41]]
val_nomr = (val-np.min(val, axis = 0))/(np.max(val, axis = 0)-np.min(val, axis = 0))
print(np.min(val, axis = 0))
print(val)
print(val_nomr)
"""
x=np.genfromtxt('dia.txt',delimiter=',')

idx=np.arange(x.shape[0]) ## Crear vector con los index de la matriz
np.random.shuffle(idx) ## Aleatoriza los valores del index de la matriz
x=x[idx,:] ## Guarda la nueva matriz aleatorizada

valores_entrada = x[:,:8] #Cargando todas las filas y las 8 primeras columnas (0 hasta 7)
valores_salida = x[:,8] #Cargando todas las filas y la ultima columna (la columna 8)
valores_entrada_normalizados = (valores_entrada-np.min(valores_entrada, axis = 0))/(np.max(valores_entrada, axis = 0)-np.min(valores_entrada, axis = 0))
X = np.array(valores_entrada_normalizados) #Conversión arreglo
Numero_Filas_Datos = X.shape[0] #Obteniendo la cantidad de elementos en una lista, es decir la cantidad de filas

N_datos_entrenamiento = int(Numero_Filas_Datos*0.7) 
N_datos_validacion = int(Numero_Filas_Datos*0.15) 
N_datos_prueba = Numero_Filas_Datos - N_datos_validacion - N_datos_entrenamiento 
matriz_normalizada = X 
vector_salida = valores_salida

matriz_entrenamiento = matriz_normalizada[:N_datos_entrenamiento,:] 
vector_salida_entrenamiento = vector_salida[:N_datos_entrenamiento] 
matriz_validacion = matriz_normalizada[N_datos_entrenamiento:(N_datos_entrenamiento+N_datos_validacion),:] 
vector_salida_validacion = vector_salida[N_datos_entrenamiento:(N_datos_entrenamiento+N_datos_validacion)] 
matriz_prueba = matriz_normalizada[N_datos_validacion:(N_datos_validacion+N_datos_prueba),:] 
vector_salida_prueba = vector_salida[N_datos_validacion:(N_datos_validacion+N_datos_prueba)] 

# ---------------------------------------------------------------------------------------------------------------

### Mediante algoritmo KNN

In [68]:
from sklearn.neighbors import KNeighborsClassifier

# Crea el modelo knn.
# Mira a los n_neighbors vecinos más cercanos.
n_neighbors = 0
# class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=1, **kwargs)
# Leer un poco más: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

flag_fin = True #Bandera de finalización
i = 0
Max_Resultado=0.0

while(flag_fin):
    i = i + 1
    n_neighbors = i
    clasificador = KNeighborsClassifier(n_neighbors)
    clasificador.fit(matriz_entrenamiento, vector_salida_entrenamiento)
    # Haga predicciones de puntos en el conjunto de prueba usando el modelo de ajuste.
    resultado = clasificador.score(matriz_validacion, vector_salida_validacion)
    print("Entrenamiento con:", i, "vecinos y con una eficiencia de:", resultado)
    if(resultado>Max_Resultado):
        Max_Resultado = resultado
        Best_Clasificador=clasificador
    if(i>=10):
        flag_fin = False

#Prueba Final:
resultado_final = Best_Clasificador.score(matriz_prueba, vector_salida_prueba)
print("El resultado final tiene una eficiencia de: ", resultado_final)

Entrenamiento con: 1 vecinos y con una eficiencia de: 0.7391304347826086
Entrenamiento con: 2 vecinos y con una eficiencia de: 0.7478260869565218
Entrenamiento con: 3 vecinos y con una eficiencia de: 0.782608695652174
Entrenamiento con: 4 vecinos y con una eficiencia de: 0.7739130434782608
Entrenamiento con: 5 vecinos y con una eficiencia de: 0.7739130434782608
Entrenamiento con: 6 vecinos y con una eficiencia de: 0.7739130434782608
Entrenamiento con: 7 vecinos y con una eficiencia de: 0.7739130434782608
Entrenamiento con: 8 vecinos y con una eficiencia de: 0.782608695652174
Entrenamiento con: 9 vecinos y con una eficiencia de: 0.7739130434782608
Entrenamiento con: 10 vecinos y con una eficiencia de: 0.7478260869565218
El resultado final tiene una eficiencia de:  0.853448275862069


# ---------------------------------------------------------------------------------------------------------------

### Mediante Perceptrón 

#### Vector de etiquetas entre -1 y 1

In [69]:
#https://www.google.com.co/search?biw=1366&bih=662&tbm=isch&sa=1&ei=wm3jWq-DPYHX5gLQx6-QCA&q=min+max+normalization&oq=minmax+&gs_l=psy-ab.3.0.0i13k1j0i13i30k1j0i30k1j0i13i30k1l7.1848.1848.0.6406.1.1.0.0.0.0.119.119.0j1.1.0....0...1c.1.64.psy-ab..0.1.118....0.vLqZciUMaXA#imgrc=a6dj_gZ1rASjKM:
vector_salida_entrenamiento = vector_salida_entrenamiento*2-1

vector_salida_validacion = vector_salida_validacion*2-1

vector_salida_prueba = vector_salida_prueba*2-1

#### Definiendo función de activación para implementar el algoritmo perceptrón

In [70]:
#Definición de la función sigmoidal
def sigmoid(x):
    return 1/(1+np.exp(-x))

#### Implementando algoritmo Perceptron para entrenamiento y validación

In [71]:
def Perceptron_Train(matriz_train, vector_etiquetas_entrenamiento, max_iteraciones = 3000, eficiencia_min = 0.80):
    contador_iteraciones = 0 #cuenta el numero de iteraciones para obtner luego una rata de error 
    contador_errores_train = 0 #cuenta los datos que fueron mal clasificados durante la iteración con la matriz de entrenamiento
    result_valid = False #Resultado de la validación, false si no valida y true si valida
    flag_culm = True #indica el estado dela iteración del algoritmo
    vector_pesos = np.random.rand(matriz_train.shape[1] + 1,) #se crea el vector de pesos inicial aleatorio
    while(flag_culm):
        contador_iteraciones = contador_iteraciones + 1;
        dato_escogido = np.random.randint(0,N_datos_entrenamiento) #Escogiendo un numero para indicar que fila de datos escoger
        g  = np.dot(vector_pesos.T,np.insert(matriz_train[dato_escogido,:],0,1)) # y = w.xT donde Y = matriz de salida, wT = Matriz de pesos
        yp = sigmoid(g)-0.5#Pasando por la funcion de activación que se le baja su offset de 0.5 
        result_comparacion = yp*vector_etiquetas_entrenamiento[dato_escogido]
        print (result_comparacion)
        #if( (yp >= 0 & vector_etiquetas_entrenamiento[dato_escogido] < 0) | (yp <= 0 & vector_etiquetas_entrenamiento[dato_escogido]> 0)):
        if result_comparacion < 0:
            contador_errores_train = contador_errores_train +1
            vector_pesos = vector_pesos + np.dot(np.insert(matriz_train[dato_escogido,:],0,1),vector_etiquetas_entrenamiento[dato_escogido]) #actualización del vector de pesos 
        result_valid = func_valid(vector_pesos,matriz_validacion, vector_salida_validacion, eficiencia_min)
        if(contador_iteraciones == max_iteraciones):
            flag_culm = False #Dejar de iterar
            print("culminacion por iteraciones")
        if(result_valid):
            print("culminacion por validacion")
            flag_culm = False #Dejar de iterar
    print("el numero de iteraciones fue: ",contador_iteraciones)
    #print("contador de errores train: ",contador_errores_train)
    print("El vector de pesos optimo W es:\n")
    for i in range(0,len(vector_pesos)):
        print("W[",i,"]=",vector_pesos[i])
    return vector_pesos

def func_valid(vector_pesos,matriz_validacion, vector_etiquetas_validacion, eficiencia_min):
    #nuestro criterio de parada será una eficiencia respecto al numero de datos analizados
    contador_errores_valid = 0
    matriz_validacion_bias = np.insert(matriz_validacion,0,1,axis=1)
    yp_validacion = np.dot(matriz_validacion_bias,vector_pesos.T)
    for i in range(0,len(yp_validacion)):
        g = sigmoid(yp_validacion[i])-0.5
        if g*vector_etiquetas_validacion[i] < 0: #clasificacion erronea
            contador_errores_valid = contador_errores_valid + 1 #para llevar el conteo de los datos clasificados erroneamente
            
    print("contador de errores validacion: ",contador_errores_valid)
    eficiencia = (len(matriz_validacion)-contador_errores_valid)/len(matriz_validacion)
    print("\nla eficiencia total en validacion fue: ",eficiencia)
    
    if eficiencia >= eficiencia_min:
        flag_valid = True
    else:
        flag_valid = False

In [73]:
vector_pesos_final = Perceptron_Train(matriz_entrenamiento, vector_salida_entrenamiento, max_iteraciones = 3000, eficiencia_min = 0.8)
#print("el vector de pesos final es: ",vector_pesos_final)

-0.36330422162018317
contador de errores validacion:  81

la eficiencia total en validacion fue:  0.2956521739130435
-0.1785975711232467
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
-0.20640492359823337
contador de errores validacion:  81

la eficiencia total en validacion fue:  0.2956521739130435
-0.1887795156352603
contador de errores validacion:  32

la eficiencia total en validacion fue:  0.7217391304347827
0.19201458916413872
contador de errores validacion:  32

la eficiencia total en validacion fue:  0.7217391304347827
0.22498999950254506
contador de errores validacion:  32

la eficiencia total en validacion fue:  0.7217391304347827
0.20286655281387506
contador de errores validacion:  32

la eficiencia total en validacion fue:  0.7217391304347827
-0.027950250153443712
contador de errores validacion:  81

la eficiencia total en validacion fue:  0.2956521739130435
-0.3527578635548041
contador de errores validacion:  31

la eficienc

0.11620666762545456
contador de errores validacion:  51

la eficiencia total en validacion fue:  0.5565217391304348
-0.13326770219002004
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.37062432905900144
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.4167144926152303
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.45493458499813627
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.4235056197185023
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.4017693521101227
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.38161863242933114
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.4203361606024412
contador de errores validacion:  34

la eficiencia tota

0.03874778972355303
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.05597740348431007
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
-0.08125097664990533
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.3718455944813178
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
-0.4054381555461856
contador de errores validacion:  25

la eficiencia total en validacion fue:  0.782608695652174
0.26073329438565707
contador de errores validacion:  25

la eficiencia total en validacion fue:  0.782608695652174
0.11334824683976985
contador de errores validacion:  25

la eficiencia total en validacion fue:  0.782608695652174
-0.0429191169811749
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
0.36697832227721844
contador de errores validacion:  34

la eficiencia tota

contador de errores validacion:  54

la eficiencia total en validacion fue:  0.5304347826086957
-0.1630852259883192
contador de errores validacion:  35

la eficiencia total en validacion fue:  0.6956521739130435
-0.051254530327834225
contador de errores validacion:  58

la eficiencia total en validacion fue:  0.4956521739130435
0.28870516418762726
contador de errores validacion:  58

la eficiencia total en validacion fue:  0.4956521739130435
-0.19057956092500428
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
0.4015486164272594
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
0.36742068360653524
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
0.15624008096801534
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
-0.35536002065404637
contador de errores validacion:  55

la eficiencia total en validacio

contador de errores validacion:  77

la eficiencia total en validacion fue:  0.33043478260869563
-0.359448125813266
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.10321538773147232
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
-0.11281197846857949
contador de errores validacion:  76

la eficiencia total en validacion fue:  0.3391304347826087
-0.12228892452065854
contador de errores validacion:  28

la eficiencia total en validacion fue:  0.7565217391304347
0.003579226017191517
contador de errores validacion:  28

la eficiencia total en validacion fue:  0.7565217391304347
-0.13272612908446735
contador de errores validacion:  33

la eficiencia total en validacion fue:  0.7130434782608696
-0.42519578344234865
contador de errores validacion:  31

la eficiencia total en validacion fue:  0.7304347826086957
-0.07593941603262161
contador de errores validacion:  76

la eficiencia total en valida

...

contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.06791615039625043
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.35689988583056925
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.30227879066637586
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.0985381692132451
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.40196122490235897
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.04699083526690295
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.04507678813415411
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.10111130879043351
contador de errores validacion:  29

la eficiencia total en validacion fu

contador de errores validacion:  26

la eficiencia total en validacion fue:  0.7739130434782608
0.2960848544664749
contador de errores validacion:  26

la eficiencia total en validacion fue:  0.7739130434782608
0.30479459871995707
contador de errores validacion:  26

la eficiencia total en validacion fue:  0.7739130434782608
0.09981330698631175
contador de errores validacion:  26

la eficiencia total en validacion fue:  0.7739130434782608
0.12486144656432108
contador de errores validacion:  26

la eficiencia total en validacion fue:  0.7739130434782608
-0.016491239502845756
contador de errores validacion:  68

la eficiencia total en validacion fue:  0.40869565217391307
-0.17299824633186722
contador de errores validacion:  38

la eficiencia total en validacion fue:  0.6695652173913044
0.21625160520483694
contador de errores validacion:  38

la eficiencia total en validacion fue:  0.6695652173913044
-0.32986916296521385
contador de errores validacion:  34

la eficiencia total en validaci

0.1858789630137449
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
-0.39604804890902984
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.2933732192086054
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.15163529484813443
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.3144977722665041
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.30500230211345103
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
0.3030981964219863
contador de errores validacion:  24

la eficiencia total en validacion fue:  0.7913043478260869
-0.4091005046499445
contador de errores validacion:  36

la eficiencia total en validacion fue:  0.6869565217391305
0.4667216315344734
contador de errores validacion:  36

la eficiencia total

contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
-0.040354734692270244
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
-0.10308051189017453
contador de errores validacion:  34

la eficiencia total en validacion fue:  0.7043478260869566
-0.4255166886188959
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
0.07314048979436916
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
0.03428954453317462
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
0.3655157376314807
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
-0.47341521668644204
contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
0.386275491289389
contador de errores validacion:  50

la eficiencia total en validacion 

la eficiencia total en validacion fue:  0.7043478260869566
-0.37975147595672953
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
0.32782171560258344
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
0.29853107418664426
contador de errores validacion:  27

la eficiencia total en validacion fue:  0.7652173913043478
-0.10709704007053567
contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
0.4771918143836701
contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
0.42491288560923623
contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
0.42582373872577417
contador de errores validacion:  50

la eficiencia total en validacion fue:  0.5652173913043478
-0.26477896633810705
contador de errores validacion:  29

la eficiencia total en validacion fue:  0.7478260869565218
0.365048228

#### Visualización de elemento final

In [74]:
matriz_prueba_bias = np.insert(matriz_prueba,0,1,axis=1) # se agrega el bias a la matriz de validación
prediccion_prueba = np.dot(matriz_prueba_bias,vector_pesos_final) #se hace una predicción del elemento a clasificar
cont_final=0
for i in range(len(prediccion_prueba)): # se recorren todos los datos de la matriz de prueba
    if ((sigmoid((prediccion_prueba[i]*vector_salida_prueba[i])))-0.5) < 0: # se evalúa el estado de la clasificación
        cont_final = cont_final+1 #se cuenta el numero de clasificaciones incorrectas   
print("la eficiencia final es: ",((len(matriz_prueba_bias)-cont_final)/len(matriz_prueba_bias))) # se obtiene la eficiencia total

la eficiencia final es:  0.7327586206896551
